In [7]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
import math
import os

In [8]:
folder = '/Users/parkerprowell/Desktop/DEV/audio_cat_ML/genres'
genre_folders = [ f.path for f in os.scandir(folder) if f.is_dir() ]

In [9]:
JSON_OUTPUT = 'mfcc_extract.json'
SAMPLE_RATE = 22050
TRACK_DURATION = 30 # measured in seconds
SAMPLES_PER_TRACK = SAMPLE_RATE * TRACK_DURATION
#_______________________________________________________________________

num_mfcc=13
n_fft=2048
hop_length=512
num_segments=5
samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)
num_mfcc_vectors_per_segment = math.ceil(samples_per_segment / hop_length)
#_______________________________________________________________________

data = {
        'genre_labels': [],
        'genre_id': [],
        'mfcc': []
    }

In [10]:
genre_labels = []
for f in genre_folders:
    genre_labels.append(f.split('/')[-1])

data['genre_labels'] = genre_labels

In [14]:
genre_labels

['pop',
 'metal',
 'disco',
 'blues',
 'reggae',
 'classical',
 'rock',
 'hiphop',
 'country',
 'jazz']

In [ ]:
for i, directory in enumerate(genre_folders):
    for j, filename in enumerate(os.listdir(directory)):
        if filename.endswith(".mp3") or filename.endswith(".wav"):
            print(f"""_____________________________________
                    File: {filename}
                    Genre: {genre_labels[i]}
                    Genre ID: {i}
                    Track #: {j}
                    _____________________________________""")
            
            file_loc = directory+'/'+filename
            #                                                         * load audio file into numpy array through librosa
            y, sr = librosa.load(file_loc, sr=SAMPLE_RATE)
            #                                                         * segment auio into 5 three second portions (artificially enlarging our dataset)
            for segment in range(num_segments):
                start = segment * samples_per_segment
                end = start + samples_per_segment
                
                mfcc = librosa.feature.mfcc(y[start:end], 
                                            sr, 
                                            n_mfcc=num_mfcc, 
                                            n_fft=n_fft, 
                                            hop_length=hop_length)
                mfcc = mfcc.T
                
                #                                                     * our data needs to be the same shape, therefor must contain a constant # of vectors
                if len(mfcc) == num_mfcc_vectors_per_segment:
                        data["mfcc"].append(mfcc.tolist())
                        data["genre_id"].append(i)
                        if segment == num_segments-1:
                            print(f'{segment+1}/{num_segments}\n{filename} Processing Completed')
                        else:
                            print(f'{segment+1}/{num_segments}')

In [6]:
with open('mfcc_extract(3,000-10sec).json', 'w') as outfile:
    json.dump(data, outfile, indent=4)

In [ ]:
data['genre_id']